In [1]:
import functools

import numpy as np
import tensorflow as tf

In [45]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      num_epochs=1,
      #label_name=LABEL_COLUMN,
      shuffle=False,
      #prefetch_buffer_size=2,
      ignore_errors=True, 
      **kwargs)
  return dataset

train_file_path = 'data/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv'
data = get_dataset(prices_path)

# TODO: Not load timestamp column
NUMERIC_FEATURES = [
    'Close',
    'Volume_(BTC)',
    'replies_sum',
    'replies_mean',
    'likes_sum',
    'likes_mean',
    'retweets_sum',
    'retweets_mean',
    'Negative_mean',
    'Negative_replies_mean',
    'Negative_likes_mean',
    'Negative_retweets_mean',
    'Neutral_mean',
    'Neutral_replies_mean',
    'Neutral_likes_mean',
    'Neutral_retweets_mean',
    'Compound_mean',
    'Compound_replies_mean',
    'Compound_likes_mean',
    'Compound_retweets_mean',
    'Polarity_mean',
    'Polarity_replies_mean',
    'Polarity_likes_mean',
    'Polarity_retweets_mean',
    'Subjectivity_mean',
    'Subjectivity_replies_mean',
    'Subjectivity_likes_mean',
    'Subjectivity_retweets_mean
]

BOOLEAN_FEATURES = [
    'no_tweets'
]

CATEGORICAL_FEATURES = []
train_file_path = 'data/all_data.csv'
#data = get_dataset(train_file_path, field_delim='\t, select_columns=NUMERIC_FEATURES+CATEGORICAL_FEATURES+BOOLEAN_FEATURES)

In [46]:
import pandas as pd
desc = pd.read_csv(train_file_path, usecols=NUMERIC_FEATURES).describe()

MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

OrderedDict([('Timestamp', <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([1.3253179e+09, 1.3253180e+09, 1.3253180e+09, 1.3253181e+09,
       1.3253181e+09], dtype=float32)>), ('Open', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4.39,  nan,  nan,  nan,  nan], dtype=float32)>), ('High', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4.39,  nan,  nan,  nan,  nan], dtype=float32)>), ('Low', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4.39,  nan,  nan,  nan,  nan], dtype=float32)>), ('Close', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4.39,  nan,  nan,  nan,  nan], dtype=float32)>), ('Volume_(BTC)', <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.45558086,        nan,        nan,        nan,        nan],
      dtype=float32)>), ('Volume_(Currency)', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 2., nan, nan, nan, nan], dtype=float32)>), ('Weighted_Price', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([4.39,  nan,  nan,  nan,  nan], dtyp

In [ ]:
CATEGORIES = {
    # 'class' : ['First', 'Second', 'Third']
}
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [ ]:
boolean_columns = []
for feature in BOOLEAN_FEATURES:
    bool_col = tf.feature_column.numeric_column(key=feature, default_value=0, dtype=tf.int8)
    boolean_columns.append(bool_col)

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numeric_columns+categorical_columns+boolean_columns)
